- join tables
- remove duplicates
- outliers / missing data
- standarize names -> coordinates to names of places
- ...

deixo aixo aqui per si pot servir despres

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import duckdb
sys.path.append('..')
from helper import *
setwd()

Objects = Objects()

from pathlib import Path

In [2]:
import geopy.geocoders
from geopy.geocoders import Nominatim
geopy.geocoders.options.default_user_agent = 'app_adsdb'

#geolocator = Nominatim(user_agent='app_adsdb/1')
geolocator = Nominatim(timeout=3)

In [3]:

def postcode(df,latitude="Latitude",longitude="Longitude"):
    Lat = np.array(df[[latitude]]).flatten()
    Long = np.array(df[[longitude]]).flatten()

    coordinates = [(Lat[i],Long[i]) for i in range(len(Lat))]#len(Lat)
    location = [geolocator.reverse(coor).raw["address"] for coor in coordinates]

    postcode = []

    for loc in location:
        if "postcode" in loc:
            postcode.append(loc["postcode"])
        else:
            postcode.append("")

    return df.assign(ZIP=list(postcode))
    


In [4]:
def drop_columns(df,alpha=0.9):
    types_df = [str(x) for x in list(df.dtypes)]
    columns_names = list(df.columns)
    missings = 0
    n = len(df)
    drop_list = []

    for i in range(len(columns_names)):

        if types_df[i] == "float64" or types_df[i] == "int32":
            missings = df[columns_names[i]].isna().sum()

        elif types_df[i] == "object":
            counts = dict(df[columns_names[i]].value_counts())
            if "unknow" in counts:
                missings = counts["unknow"]

        if missings/n > alpha:
            drop_list.append(columns_names[i])

    print(drop_list)
    return df.drop(drop_list,axis=1)


In [5]:
# numeric -> np.nan
# object -> "unknow"
def standarMissings(df):
    types_df = [str(x) for x in list(df.dtypes)]
    columns_names = list(df.columns)
    
    for i in range(len(columns_names)):
        
        if types_df[i] == "float64" or types_df[i] == "int32":
            df[columns_names[i]] = df[columns_names[i]].replace(-999, np.nan)

        elif types_df[i] == "object":
            df[columns_names[i]] = df[columns_names[i]].replace(np.nan, "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("", "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("xxxxxx", "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("nan", "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("......", "unknow")
            df[columns_names[i]] = df[columns_names[i]].replace("NOT AVAILABLE", "unknow")
            

In [6]:
def formatted2trusted():
    
    # list of tables already processed:
    processed_table = []
    with open('logs/processed_data_trusted.txt', "r") as f0:
        for line in f0:
            processed_table.append(line.rstrip())
    
    # process datasets:
    for datasource in Objects:
        id = datasource["id"]

        # conect to DB
        con = duckdb.connect(database="data/formatted/db_{}.db".format(id), read_only=True)

        # take all tables names
        tables = con.execute("SHOW TABLES").fetchall()
        tables_names = [x[0] for x in tables]

        # tables to process:
        tables_not_processed = [table for table in tables_names if not table in processed_table]
        if len(tables_not_processed) == 0:
            continue
        
        # list of all tables in dataframe format
        dataframes = []
        for table in tables_not_processed:
            dataframes.append(con.execute("SELECT * FROM {}".format(table)).fetchdf())

        # close database
        con.close()

        # merge all dataframes
        df = pd.concat(dataframes)
        standarMissings(df)
        df.drop_duplicates()
        df = drop_columns(df)

        #column_names = list(df.columns.values)
        #if "ZIP" not in column_names:
            #print("No ZIP",len(df))
            #df2 = df.head(100)
            #postcode(df2,latitude="lat",longitude="long")

        my_file = Path("data/trusted/db_{}.db".format(id))
        if my_file.is_file():
            con = duckdb.connect(database="data/trusted/db_{}.db".format(id), read_only=False)
            df_original = con.execute("SELECT * FROM {}".format(id)).fetchdf()
            df = pd.concat([df,df_original])
            df.drop_duplicates()#CREATE OR REPLACE TABLE
            df = drop_columns(df)
            con.execute("CREATE OR REPLACE TABLE {} AS SELECT * FROM df".format(id))
            con.close()
        
        else:
            con = duckdb.connect(database="data/trusted/db_{}.db".format(id), read_only=False)
            con.execute("CREATE TABLE {} AS SELECT * FROM df".format(id))
            con.close()

        

        if os.path.exists("scripts/trusted_{}.py".format(id)):
            os.system("python3 ./scripts/trusted_{}.py".format(id))

        with open('logs/processed_data_trusted.txt', "a") as f0:
            for table in tables_not_processed:
                f0.write(table + "\n")

    

In [7]:
formatted2trusted()

[]
[]
['ZIP4', 'TTL_STAFF']


In [8]:
con = duckdb.connect(database="data/trusted/db_schools.db", read_only=False)
df = con.execute("SELECT * FROM schools;").fetchdf()
con.close()
df["LEVEL_"].head(10)

0            preschool
1    elementary_school
2    elementary_school
3            preschool
4    elementary_school
5            preschool
6    elementary_school
7              unknown
8    elementary_school
9        middle_school
Name: LEVEL_, dtype: object

In [44]:
id = "schools"
con = duckdb.connect(database="data/trusted/db_{}.db".format(id), read_only=True)
df = con.execute("SELECT * FROM {}".format(id)).fetchdf()
#df.describe()
df.head()


,X,Y,OBJECTID,NCESID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,VAL_METHOD,VAL_DATE,WEBSITE,LEVEL_,ENROLLMENT,ST_GRADE,END_GRADE,DISTRICTID,FT_TEACHER,SHELTER_ID
0,-120.676576,38.732865,92398,60717000656,CAMINO ELEMENTARY,3060 SNOWS RD.,CAMINO,CA,95709,unknow,...,IMAGERY/OTHER,2016-02-09,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,423,KG,08,607170,23,unknow
1,-122.080806,37.967529,92399,62637003977,VALHALLA ELEMENTARY,530 KIKI DR.,PLEASANT HILL,CA,94523,1717,...,IMAGERY,2016-02-09,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,542,KG,05,626370,21,10813228
2,-122.025277,37.934626,92400,62637003986,YGNACIO VALLEY HIGH,755 OAK GROVE RD.,CONCORD,CA,94518,2899,...,IMAGERY,2016-02-09,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,3,1134,09,12,626370,54,10813162
3,-82.034343,33.400503,92403,130438001557,ROLLINS ELEMENTARY SCHOOL,2160 MURA DR,AUGUSTA,GA,30906,unknow,...,IMAGERY/OTHER,2016-03-22,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,444,PK,05,1304380,27,unknow
4,-100.040761,37.761009,92409,200558000887,LINN ELEMENTARY,1900 W LINN ST,DODGE CITY,KS,67801,unknow,...,IMAGERY/OTHER,2016-04-08,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,468,KG,05,2005580,24,unknow


In [45]:
con.execute("SELECT * FROM profiling").fetchdf()

,Columns,Missings
0,X,2
1,Y,2
2,OBJECTID,0
3,NAME,0
4,ADDRESS,0
5,CITY,0
6,STATE,0
7,ZIP,0
8,ZIP4,196368
9,TELEPHONE,105996


In [ ]:
import re
Col = sorted(df['COUNTY'].apply(lambda x: str(x)).unique())
print(len(Col))
"""
for i in Col:
    if not re.match("[0123456789]*", i) :
        print(i)

"""
Col.append("unknow")
Col2 = [x for x in Col if len(x) < 15]
print(len(Col2))
print("unknow" in Col2)
Col2
